# 4장 - 신경망 학습
----------------------
----------------------
학습: 훈련 데이터로부터 가중치 매개변수의 최적값을 자동으로 획득하는 것<br>
손실함수: 신경망이 학습할 수 있도록 해주는 지표<br>

----------------------
### 데이터에서 학습한다!
신경망은 이미지를 '있는 그대로' 학습한다.<br>
기계학습: 특징을 사람이 설계 VS 딥러닝(신경망): 이미지에 포함된 중요한 특징까지도 '기계'가 스스로 학습<br>
딥러닝 = 종단간학습: 처음(입력)부터 끝(출력)까지 사람의 개입 없이 얻는다<br>
<br>
![신경망 설명](그림자료들/chapter4-1.jpg)<br>
<br>
훈련 데이터와 시험 데이터를 나누는 이유<br>
==> 우리가 원하는 것은 범용적으로 사용할 수 있는 모델이기 때문<br>
==> 즉, 모델의 범용 능력(아직 보지 못한 데이터-훈련 데이터에 포함되지 않는 데이터-로도 문제를 올바르게 풀어내는 능력)을 보기 위함<br>

----------------------
### 손실 함수
손실 함수는 신경망 성능의 '나쁨'을 나타내는 지표이다.<br>
일반적으로 사용되는 함수는 다음과 같다.<br>
1. 오차제곱합 (가장 많이 쓰이는 손실 함수)
2. 교차 엔트로피 오차

----------------------
원-핫 인코딩: 한 원소만 1로 하고 나머지는 0으로 나타내는 표기법

In [57]:
# 오차제곱합 구현
import numpy as np

def sum_squares_error(y, t): # y=각 원소의 출력(추정) 값, t=정답 레이블(참 값)
    return 0.5 * np.sum((y-t)**2)

In [58]:
# 예시1
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0] # t값 가정
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
sum_squares_error(np.array(y), np.array(t))

0.09750000000000003

In [59]:
# 예시2
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0] # t값 가정
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
sum_squares_error(np.array(y), np.array(t))

0.5975

In [60]:
# 교차 엔트로피 오차 구현
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta)) # np.log() 함수에 0을 입력하면 마이너스 무한대(-inf)가 되어 계산이 불가능해지므로 아주 작은 값(delta)을 더해서 절대 0이 되지 않도록 함

In [61]:
# 예시1
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0] # t값 가정
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
cross_entropy_error(np.array(y), np.array(t))

0.510825457099338

In [62]:
# 예시2
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0] # t값 가정
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
cross_entropy_error(np.array(y), np.array(t))

2.302584092994546

미니배치 학습: 신경망 학습에서 훈련 데이터로부터 일부(미니배치)만 골라 학습을 수행하는 것

In [63]:
# 훈련 데이터 불러오기
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

print(x_train.shape)
print(t_train.shape)

(60000, 784)
(60000, 10)


In [64]:
# 훈련 데이터에서 지정한 수의 데이터를 무작위로 골라내는 코드 작성
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

In [65]:
np.random.choice(60000, 10)

array([47482, 34033, 10522,  7276, 11914, 20912, 48186, 44099, 10788,
        5852])

In [66]:
# (배치용) 교차 엔트로피 오차 구현
# 정답 레이블이 원-핫 인코딩일 때
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    
    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + 1e-7)) / batch_size

In [67]:
# (배치용) 교차 엔트로피 오차 구현
# 정답 레이블이 원-핫 인코딩이 아닌 '2'나 '7'처럼 숫자 레이블일 때
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

경사법(경사하강법): 기울기를 잘 이용해 함수의 최솟값을 찾으려는 것

In [68]:
# 기울기 구현
def numerical_gradient(f, x):
    h = 1e-4
    grad = np.zeros_like(x) # x와 형상이 같은 배열을 생성

    for idx in range(x.size):
        tmp_val = x[idx]
        
        # f(x+h) 계산
        x[idx] = tmp_val + h
        fxh1 = f(x)

        # f(x-h) 계산
        x[idx] = tmp_val - h
        fxh2 = f(x)

        grad[idx] = (fxh1 - fxh2) / (2*h)
        x[idx] = tmp_val # 값 복원
    
    return grad

# 경사 하강법 구현
def gradient_descent(f, init_x, lr=0.01, step_num=100): # f=최적화하려는 함수, init_x=초깃값, lr=learning rate, step_num=경사법에 따른 반복 횟수
    x = init_x

    for i in range(step_num):
        grad = numerical_gradient(f, x) # 함수의 기울기 구하기 -> 그 기울기에 학습률을 곱한 값으로 갱신하는 처리를 step_num번 반복
        x -= lr * grad
    
    return x

In [69]:
# 신경망에서의 기울기 구현
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.functions import softmax, cross_entropy_error
from common.gradient import numerical_gradient

class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2,3) # 정규분포로 초기화

    def predict(self, x):
        return np.dot(x, self.W)
    
    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)
        return loss

In [70]:
# 2층 신경망 클래스 구현하기
import sys, os
sys.path.append(os.pardir)
from common.functions import *
from common.gradient import numerical_gradient

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01): # input_size=입력층의 뉴런 수, hidden_size=은닉층의 뉴런 수, output_size=출력층의 뉴런 수
        # 가중치 초기화
        self.params = {} # 신경망의 매개변수를 보관하는 딕셔너리 변수(인스턴스 변수)
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size) # params['W1']=1번째 층의 가중치
        self.params['b1'] = np.zeros(hidden_size) # params['b1']=1번째 층의 편향
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

    def predict(self, x): # 예측(추론)을 수행하는 함수, x=이미지 데이터
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']

        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)

        return y
    
    def loss(self, x, t): # 손실 함수의 값 구하는 함수, x=입력 데이터, t=정답 레이블
        y = self.predict(x)

        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t): # 정확도 구하는 함수
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self, x, t): # 가중치 매개변수의 기울기 구하는 함수
        loss_W = lambda W: self.loss(x, t)

        grads = {} # 기울기를 보관하는 딕셔너리 변수(numerical_gradient() 메서드의 반환 값)
        grads['W1'] = numerical_gradient(loss_W, self.params['W1']) # grads['W1']=1번째 층의 가중치의 기울기
        grads['b1'] = numerical_gradient(loss_W, self.params['b1']) # grads['b1']=1번째 층의 편향의 기울기
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

        return grads

In [ ]:
# 미니배치 학습 구현하기
import numpy as np
from dataset.mnist import load_mnist
#from two_layer_net import TwoLayerNet

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

train_loss_list = []

iters_num = 10000 # 반복 횟수
train_size = x_train.shape[0]
batch_size = 100 # 미니배치 크기
learning_rate = 0.1

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for i in range(iters_num):
    # 미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    # 기울기 계산
    grad = network.numerical_gradient(x_batch, t_batch)

    # 매개변수 갱신
    for key in ('W1', 'b2', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]

    # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

In [ ]:
# 미니배치 학습 구현하기 => 시험 데이터로 평가하기
import numpy as np
from dataset.mnist import load_mnist
#from two_layer_net import TwoLayerNet

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

train_loss_list = []
train_acc_list = []
test_acc_list = []

# 1에폭당 반복 수
iter_per_epoch = max(train_size / batch_size, 1)

iters_num = 10000 # 반복 횟수
train_size = x_train.shape[0]
batch_size = 100 # 미니배치 크기
learning_rate = 0.1

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for i in range(iters_num):
    # 미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    # 기울기 계산
    grad = network.numerical_gradient(x_batch, t_batch)

    # 매개변수 갱신
    for key in ('W1', 'b2', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]

    # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    # 1에폭당 정확도 계산
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc | " + str(train_acc) + ", " + str(test_acc))